In [ ]:
!pip install pymp-pypi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for pymp-pypi: filename=pymp_pypi-0.5.0-py3-none-any.whl size=10339 sha256=473fde4521d50d3c64fcb49b8451f5fce26a371bb956d77aa7d36faa0b221ed6
  Stored in directory: /root/.cache/pip/wheels/22/2a/4e/d49c406bb5eb2c04b424940de41d40b3b1481c31b9f93a13c1
Successfully built pymp-pypi


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import networkx as nx
import pymp
import threading
import time


G_tmp = nx.read_edgelist('/content/drive/MyDrive/Colab Notebooks/web-Google.txt', create_using = nx.DiGraph)
print(nx.info(G_tmp))

# To begin with, let's simplify our analysis by ignoring the dangling nodes and the disconnected components in the original graph.
c = sorted(nx.weakly_connected_components(G_tmp), key=len, reverse=True)
wcc_set = c[0]
G = G_tmp.subgraph(wcc_set)
print(nx.info(G))

DiGraph with 875713 nodes and 5105039 edges
DiGraph with 855802 nodes and 5066842 edges


In [ ]:
tele_p = 0.15
A = nx.adjacency_matrix(G)
A = A.T

n = A.shape[0]
A = A.tocsr()
print(type(A))
A = A.astype('float64')

# A = np.array(A, dtype = 'float64'
out_degrees = np.array([G.out_degree[x] for x in list(G.nodes)], dtype='float64')
A.data *= (1-tele_p) / np.take(out_degrees, A.indices)

r =  np.ones((n,1))
error = 1

<class 'scipy.sparse.csr.csr_matrix'>


In [ ]:
start = time.time()
for _ in range(10):
#     if _ % 100 == 0:
#         print("iter", _)
    r_new = A @ r
    r_new += np.sum(r) * tele_p / n
#     r_new /= np.sum(r_new)
    error = np.sum(np.absolute(r - r_new))
    r = r_new
    if error < 1e-6:
        break

r /= np.sum(r)
r = r.reshape(n)

print("part of my result\n", r[:50])
truth = nx.pagerank(G, alpha=0.85, max_iter = 100)
truth = np.array(list(truth.values()), dtype='float64')
print("part of networkx's result\n",truth[:50])
end = time.time()
print(end-start)

part of my result
 [3.85815304e-05 4.00701357e-05 9.17296455e-06 4.06048437e-05
 3.92275364e-05 7.56980779e-06 1.15057046e-05 3.57252804e-06
 4.17035960e-06 4.17035960e-06 5.12689231e-06 2.88231421e-06
 3.01044209e-06 3.21045624e-06 8.51999968e-06 3.56205012e-06
 1.08455058e-06 1.47372821e-04 3.43878437e-04 7.88711799e-06
 7.80079949e-06 1.09102265e-06 5.20667230e-06 4.85943119e-06
 3.79799035e-06 7.12106861e-06 4.03820224e-06 3.52343295e-06
 1.58183203e-05 3.76417755e-06 3.73410106e-06 3.73410106e-06
 3.73410106e-06 4.74390242e-06 2.36847042e-06 9.98521594e-06
 1.26104413e-05 3.43715028e-04 9.60930635e-06 9.48328957e-06
 9.76852109e-06 9.85223140e-06 9.82741681e-06 9.66320395e-06
 9.71427373e-06 1.93430593e-06 1.92770297e-06 2.06275280e-06
 5.27340743e-06 1.92770297e-06]
part of networkx's result
 [2.83545918e-05 3.17140006e-05 7.15313456e-06 3.18433656e-05
 3.10020184e-05 5.45502590e-06 1.15575220e-05 3.07694855e-06
 2.69228185e-06 2.69228185e-06 5.17349351e-06 2.05934593e-06
 2.0815

In [ ]:
# using pymp to parallelize the code
def pagerank_pymp(A, n, tele_p, r, error):
    for _ in range(10):
        r_new = A @ r
        r_new += np.sum(r) * tele_p / n
        error = np.sum(np.absolute(r - r_new))
        r = r_new
        if error < 1e-6:
            break
    return r

# using threading to parallelize the code
def pagerank_threading(A, n, tele_p, r, error):
    for _ in range(10):
        r_new = A @ r
        r_new += np.sum(r) * tele_p / n
        error = np.sum(np.absolute(r - r_new))
        r = r_new
        if error < 1e-6:
            break
    return r

def main():
    # pymp
    start = time.time()
    r =  np.ones((n,1))
    error = 1
    with pymp.Parallel(4) as p:
        for i in p.range(4):
            r = pagerank_pymp(A, n, tele_p, r, error)
    end = time.time()
    print("pymp time:", end - start)
    
    # threading
    
    start = time.time()
    r =  np.ones((n,1))
    error = 1
    threads = []
    for i in range(4):
        t = threading.Thread(target=pagerank_threading, args=(A, n, tele_p, r, error))
        threads.append(t)
        t.start()
    for t in threads:
        t.join()
    end = time.time()
    
    print("threading time:", end - start)

In [ ]:
main()

pymp time: 1.6641438007354736
threading time: 1.4801280498504639
